In [1]:
import glob
from skimage import color
from skimage import io
import skimage
import numpy as np
from skimage.util import img_as_float
import pandas as pd

In [2]:
def get_filelist(img_loc):
    filelist = glob.glob(img_loc+'*.jpg')
    filelist.sort()
    return filelist

In [3]:
filelist=get_filelist("/home/sirzechlucifer/Work/CVPR_graphs/ISIC2018_input/")

In [4]:
def preprocessing(filelist):
    input_shape = [10015, 224, 224]
    X_orig = np.empty(input_shape)
    j=0
    for i in filelist:
        img = color.rgb2gray(io.imread(i))
        image = img_as_float(img)
        image=skimage.transform.resize(image, (224,224), mode='constant')
        X_orig[j]=image
        j=j+1
    return X_orig

In [5]:
X_orig=preprocessing(filelist)

/home/sirzechlucifer/anaconda3/lib/python3.7/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [6]:
X_orig.shape

(10015, 224, 224)

In [7]:
def get_Y_orig(labels_loc):
    #output_list=list()
    Y_df = pd.read_csv(labels_loc)
    Y_df=Y_df.iloc[:,1:]
#     for index,row in Y_df.iterrows():
#         if row["MEL"]==1:
#             output_list.append(0)
#         elif row["NV"]==1:
#             output_list.append(1)
#         elif row["BCC"]==1:
#             output_list.append(2)
#         elif row["AKIEC"]==1:
#             output_list.append(3)
#         elif row["BKL"]==1:
#             output_list.append(4)
#         elif row["DF"]==1:
#             output_list.append(5)
#         elif row["VASC"]==1:
#             output_list.append(6)
    return Y_df

In [8]:
labels_loc = '/home/sirzechlucifer/Work/CVPR_graphs/ISIC2018_target/ISIC2018_Task3_Training_GroundTruth.csv'

Y_orig = get_Y_orig(labels_loc)

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten

In [12]:
model = Sequential()
model.add(Conv2D(128, kernel_size=3, activation='relu', input_shape=(224,224,1)))
model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(7, activation='softmax'))

In [13]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
X_orig=X_orig.reshape((10015,224,224,1))

In [ ]:
model.fit(X_orig, Y_orig, epochs=50)